In [52]:
import os 
import time 
import requests 
from selenium import webdriver 
import numpy as np
import random
from bs4 import BeautifulSoup 
from urllib.parse import quote_plus
import pandas as pd


def scroll_pause_time():
    num = np.random.randint(5)+1
    scroll_pause_time = num
    
    return scroll_pause_time

def tweet_crawler(keyword) :
    global Crawling_Contents
    Tweet_Contents =pd.DataFrame()
    #path = os.path.realpath(os.path.dirname(__file__))
    #dataFile = open(os.path.join(path,"{}".format(keyword)), encoding='utf-8-sig', mode ='at')
    

    
    
    #My tweeter account 
    Tweeter_Account = {'id':"****", "pw":'*****'}
    
   
    
    options = webdriver.ChromeOptions()
    #options.add_argument('headless')
    
    
    # path 지정 
    path = ["크롤링_작업중"]
    path = [dir.replace("/", "") for dir in path]
    path = os.path.join(os.path.abspath(''),*path)
    
    driver = webdriver.Chrome(executable_path=os.path.join(path,'chromedriver') , chrome_options=options)
    
    #base_url = 'https://mobile.twitter.com/search?q=%%'
    #base_url = 'https://mobile.twitter.com/search?q=%'
    #base_url2 = '&src=typed_query&f=live'
    #quote_keyword = quote_plus(keyword)
    
    #keyword_URL = base_url+quote_keyword+base_url2
    
    
    #트위터 로그인 
    
    driver.get('https://mobile.twitter.com/login?lang=ko')
    time.sleep(3)
    
    user_id = driver.find_element_by_name("session[username_or_email]")
    user_pw = driver.find_element_by_name("session[password]")
    
    user_id.send_keys(Tweeter_Account['id'])
    user_pw.send_keys(Tweeter_Account['pw'])
    driver.implicitly_wait(3)
    
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[2]/form/div/div[3]/div/div/span/span').click()
    driver.implicitly_wait(3)
    time.sleep(3)
    
    
    
    
    wait = WebDriverWait(driver, 10)
    search_input = wait.until(ec.visibility_of_element_located((By.XPATH, "//div/input[@data-testid='SearchBox_Search_Input']")))

    search_input.clear() 
    search_input.send_keys(keyword + Keys.ENTER)
    
    time.sleep(3)
    
    #최신순
    driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[1]/div[2]/nav/div/div[2]/div/div[2]/a/div').click()
    
    time.sleep(3)
    # 마지막 스크롤 지정 
    last_height= 0 
    
    # 수집할 객체 생성 
    Crawling_Contents=dict()
    Crawling_Contents['content'] = []
    Crawling_Contents['date'] = []

    
    
    while True : 
        #화면 최하단으로 스크롤 다운 
        
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        #페이지 로드 대기 
  
        
        time.sleep(scroll_pause_time())
        # 다시 절반을 더 내림 
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight-50);')
        time.sleep(scroll_pause_time())
        
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height ==last_height : 
            break 
            
        
        last_height = new_height 
        
        
        post_element_xpath = '//div/div/article/div/div'
        
        post_list = driver.find_elements_by_xpath(post_element_xpath) 
        post_text = [x.text for x in post_list]
    
        for i in post_text: 
            
            try:
                date = i.split('\n')[3]
                text = i.split('\n')[4]

                Crawling_Contents['content'].append(text)
                Crawling_Contents['date'].append(date)

            except: 
                pass 
            

    Tweet_Contents['date'] = Crawling_Contents['date']
    Tweet_Contents['Content'] = Crawling_Contents['content']  

    Tweet_Contents.to_csv('Tweet_Contents.csv', encoding='utf-8-sig')
    
    return Tweet_Contents
    
    

In [ ]:
tweet_crawler('키워드 입력')

In [55]:
# 무한 스크롤이므로 중간에 멈추고 따로 수집하였음 

Tweet_Contents =pd.DataFrame()

Tweet_Contents['date'] = Crawling_Contents['date']
Tweet_Contents['Content'] = Crawling_Contents['content']  

Tweet_Contents.to_csv('Tweet_Contents.csv', encoding='utf-8-sig')